# Índices Educación Superior Periodo 2005-2023

## I. Introducción
La educación es uno de los pilares de la sociedad, por tanto se hace interesante comprender cómo se relacionan algunos de sus indicadores y de que forma influyen unos con otros ¿existirá una relación entre el puntaje de corte de una carrera y su costo en arancel? ¿se pueden agrupar carreras y universidades según su información de matrícula?

Se posee una base de datos histórica de educación superior que cuenta con registros desde 2005 a 2023 y que se explotará a continuación.

## II. Exploración de datos

### Distribuciones importantes

En primer lugar, se decide contabilizar el porcentaje de registros correspondientes a carreras de pregrado y postgrado, resultando en un 83.8% y 16.2% respectivamente.

![Distribución de registros con respecto a la columna Pregrado/Posgrado](./outputs/dist.png)


Por otra parte, la distribución de estudiantes por regiones es la siguiente:

![Distribución de estudiantes por regiones](./outputs/count_students.png)

Aunque pudiese ser predecible, se evidencia una alta cantidad de estudiantes en la Región Metropolitana, seguida por Valparaíso y Biobío. 

En cuanto a la cantidad de estudiantes por tipo de carrera, se observa que la mayoría de los estudiantes se encuentran en carreras de tipo técnicas, seguidas por profesional con licenciatura y carreras profesionales.

![Distribución de estudiantes por tipo de carrera](./outputs/count_students_degree.png)

Agrupando las carreras por sector de conocimiento, se puede observar una gran presencia de estudiantes en carreras tecnológicas, seguidas por el área de administración y comercio. En tercer puesto, se encuentra el área de salud que es casi equiparada por el área de educación.

![Distribución de estudiantes por sector de conocimiento](./outputs/count_students_area.png)

Por el lado de las estadísticas relacionadas al ingreso a estas carreras se tiene el promedio del puntaje obtenido en las pruebas de matemáticas y lenguaje. 
Los resultados registrados entre los años 2005 y 2019 corresponden a la prueba de selección universitaria PSU, con un puntaje mínimo de 150 puntos y un máximo de 850 puntos. Los siguientes 3 años corresponden a resultados de la prueba de transición universitaria PTU, mientras que el último año registrado corresponde a la nueva prueba de acceso a la educación superior PAES, cuyo rango de puntajes va desde los 100 hasta los 1.000 puntos.

A continuación se observa la distribución de los puntajes, donde se puede apreciar que en el caso de la PSU la mayoría de los estudiantes obtienen puntajes entre 400 y 700 puntos en ambas pruebas (esto ya que la escala de cada una de las pruebas de la PSU estaba estandarizado, con una media de 500 puntos y una desviación estándar de 110).


![Distribución de puntajes PSU](./outputs/psu.png)

![Distribución de puntajes PSU](./outputs/paes.png)

### Análisis de las columnas

Desde un punto de vista más detallado, se analizarán las columnas de la base de datos para determinar cuáles son las más relevantes para el análisis y cuáles podrían ser descartadas.

* Año: Se observa que la base de datos cuenta con registros desde el año 2005 hasta el año 2023. Este atributo es numérico y el tipo de dato de esta columna es de tipo entero.
* Cód. Institución: Corresponde al código que identifica a la institución de educación superior, por lo que es un atributo numérico. El tipo de dato de esta columna es de tipo entero, no requiriendo ningún tipo de transformación. Podría ser útil para realizar agregación de datos.
* Nombre Institución: Atributo cualitativo nominal que corresponde al nombre de la institución de educación superior. El tipo de dato de esta columna es de tipo string, no requiriendo ningún tipo de transformación. Podría ser útil para realizar agregación de datos.
* Tipo Institución: Corresponde al tipo de institución de educación superior, lo que lo convierte en un atributo cualitativo. El tipo de dato de esta columna es de tipo string, no requiriendo ningún tipo de transformación, aunque al ser un atributo cualitativo podría ser ordinal y por tanto ser mapeado. Podría ser útil para realizar agrupaciones de datos.
* Clasificación1 a Clasificación6: Son atributos cualitativo y, al igual que el "Tipo Institución" se les puede considerar ordinal. El tipo de dato de esta columna es de tipo string.
Dado que podrían ser categorías importantes, se analizan sus datos para determinar a qué corresponden. 

  - Clasificación 2: Corresponde al tipo de universidad: si es CRUCH, Privada, Instituto. FFAA, etc.
  - Clasificación 1: Es una versión menos granular de la clasificación 2, por lo que se decide eliminarla.
  - Clasificación 3: Acreditación de la carrera (Acreditada, No Acreditada)
  - Clasificación 4: Si es una organización autónoma, cerrada, de supervisión, licenciamiento, etc.
  - Clasificación 5: Si está o no adscrita a gratuidad
  - Clasificación 6: Subsistema al que pertenece la carrera

* Nombre de la Sede: Atributo cualitativo nominal que corresponde al nombre de la sede de la institución de educación superior. El tipo de dato de esta columna es de tipo string, no requiriendo ningún tipo de transformación. Podría ser útil para realizar agregación o agrupación de datos.
* Comuna donde se imparte la carrera o programa y Nombre Region: Atributos cualitativo nominales que corresponden al nombre de la comuna donde se imparte la carrera o programa y su región. El tipo de dato de estas columnas son de tipo string, no requiriendo ningún tipo de transformación. Podrían ser útil para realizar agregación o agrupación de datos.
* Orden Geográfico de la Región (Norte a Sur): Atributo cuantitativo que corresponde al número de la región, siendo un número entero. El tipo de dato de esta columna es de tipo entero, no requiriendo ningún tipo de transformación.
* Cód. Carrera: Corresponde al código que identifica a la carrera, por lo que es un atributo numérico. El tipo de dato de esta columna es de tipo entero, no requiriendo ningún tipo de transformación. Podría ser útil para realizar agregación de datos.
* Carrera Genérica, Nombre Programa y Mención o Especialdad: Información cualitativa nominal que corresponde al nombre de la carrera o programa y su mención/especialidad en caso de contar con una. El tipo de dato de estas columnas es de tipo string.
* Horario: Atributo cualitativo nominal que corresponde al horario en que se imparte la carrera o programa (Diurno o Vespertino). El tipo de dato de esta columna es de tipo string. 
* Tipo Programa: Sus valores son Programa Regular y Programa Especial (por lo que es un atributo cualitativo nominal). El programa especial aplica para programas que permiten el perfeccionamiento en competencias específicas, por lo que su duración puede ser menor a la de una carrera con programa regular o tratarse de programas de formación de las fuerzas armadas. El tipo de dato de esta columna es de tipo string.
* Area Conocimiento: Atributo cualitativo nominal que corresponde al área de conocimiento de la carrera o programa. El tipo de dato de esta columna es de tipo string. Podría ser útil para realizar agrupación de datos.
* idgenerocarrera: Atributo cuantitativo que corresponde al código que identifica al género de la carrera. El tipo de dato de esta columna es de tipo entero.
* Tipo Carrera: Atributo cualitativo nominal que corresponde al tipo de carrera (Técnica, Profesional, Profesional con Licenciatura, etc.). El tipo de dato de esta columna es de tipo string. Podría ser útil para realizar agrupación de datos.
* IngresoDirecto: Atributo cualitativo nominal que corresponde a si la carrera o programa permite el ingreso directo o no. El tipo de dato de esta columna es de tipo string.
* Año Inicio Actividades: Atributo cuantitativo que corresponde al año en que la carrera o programa inició sus actividades. El tipo de dato de esta columna es de tipo entero.
* Nombre del Campus: Atributo cualitativo nominal que corresponde al nombre del campus donde se imparte la carrera o programa. El tipo de dato de esta columna es de tipo string. Podría ser útil para realizar agrupación o agregación de datos.
* Duración (en semestres): Atributo cuantitativo que corresponde a la duración de la carrera o programa en semestres. El tipo de dato de esta columna es de tipo entero.
* Cód. Campus y Cód. Sede: Corresponden a los códigos que identifican al campus donde se imparte la carrera o programa, por lo que es un atributo numérico. El tipo de dato de estas dos columnan es de tipo entero, no requiriendo ningún tipo de transformación. Podrían ser útiles para realizar agregación de datos.
* Título y Grado Académico: Atributo cualitativo nominal que corresponde al título y grado que otorga la carrera o programa. El tipo de dato de esta columna es de tipo string. Podría ser útil para realizar agrupación de datos.
* Máximo Puntaje, Promedio Puntaje y Mínimo Puntaje (promedio matemáticas y lenguaje): Atributos cuantitativos que corresponden al puntaje máximo y mínimo de ingreso a la carrera o programa, así como el promedio de estos. El tipo de dato de estas columnas es de tipo entero.
* Puntaje de corte, (primer y último seleccionado y promedio de la carrera): Atributos cuantitativos que corresponden al puntaje de corte de la carrera o programa, así como el primer y último seleccionado y el promedio de estos. El tipo de dato de estas columnas es de tipo entero.
* Máximo Puntaje, Promedio Puntaje y Mínimo Puntaje (NEM): Atributos cuantitativos que corresponden al puntaje máximo y mínimo de notas de enseñanza media de los seleccionados, así como el promedio de estos. El tipo de dato de estas columnas es de tipo entero.
* Máximo Puntaje, Promedio Puntaje y Mínimo Puntaje (Ranking): Similar al caso anterior, pero para el ranking de notas de enseñanza media. El tipo de dato de estas columnas es de tipo entero.
* N° Alumnos 

A partir de esto, se observa que hay columnas que no nos entregan información relevante, como el orden geografico de la región, el Código SIES o el idgenerocarrera. 

También la información referente a puntajes mínimos y máximos de ingreso es insuficiente (tienen una alta tasa de nulos), pero se cuenta con más valores para la columna de 'Promedio Puntaje (promedio matemáticas y lenguaje)' de las pruebas de ingreso. Las menciones o especialidades no están presentes en todas las carreras y la columna con el nombre del campus tiene una tasa de nulos del 57%, por lo que tampoco se considerarán en el experimento.

## III. Preguntas de interés
Preguntas a que se podrían resolver usando la minería de datos:
1. ¿Es posible predecir el puntaje de corte esperado para el próximo año a partir de las tendencias históricas?
2. ¿Qué factores determinan el arancel de una carrera?
3. ¿Existen universidades con características comunes? 
4. ¿Se pueden agrupar instituciones/carreras usando estas características? ¿Acaso tienen perfiles similares de aranceles y puntajes de ingreso?

Resulta interesante construir un predictor de puntajes de corte creado con datos reales, y no con simples estimaciones, o analizar el perfil de las instituciones de acuerdo a las características que se puedan extraer a partir de los atributos de la base de datos, ya que mediante simple análisis visual de las bases no siempre es directa la relación entre características de miles de datos disponibles.

Estas 4 preguntas pueden ser catalogadas en 2 grupos: preguntas de predicción y preguntas de agrupamiento (clustering).

A partir del estudio inicial del dataset se cree que hay datos suficientes para responder el primer grupo de preguntas antes planteadas, por lo tanto, se decide buscar más información en otras fuentes de datos.

## IV. Propuesta de Metodología Experimental
Considerando que nuestras preguntas serán estudiadas en dos categorías, se propone las siguientes metodologías:

**Predicción:**
- Realizar preprocesamiento: Botar columnas irrelevantes, rellenar nulos, etc.
- Fijar una carrera/clase de carrera específica: Considerando la cantidad de carreras existentes, se propone fijar una carrera específica para realizar la predicción de puntaje de corte.
- Usar modelos de regresión para predecir el puntaje de corte anual: Se debe determinar qué tipo de regresión conviene usar en función de los datos: lineal, polinomial, etc.
- Evaluar el modelo y ver qué features son importantes

**Clustering:**
- Preprocesamiento: La etapa de preprocesamiento se divide en 6 pasos
    - Botar filas irrelevantes
    - Botar columnas irrelevantes
    - Botar columnas con muchos nulos
    - Imputar nulos
    - Mapear columnas cualitativas ordinales
    - Aplicar OneHotEncoder a columnas cualitativas nominales
    - Unir ambos conjunto de datos
- Determinar número óptimo de clusters
- Usar algoritmos de clustering: K-means, DBSCAN, etc.
- Reducción de dimensiones: Se puede usar PCA para reducir la dimensionalidad de los datos y así poder visualizarlos en 2D
- Evaluación y análisis: Se debe evaluar qué tan bien se agrupan los datos y analizar los grupos obtenidos para determinar si tienen sentido o no.

## V. Experimentación

### Predicción de puntaje de corte

### Clustering de carreras e instituciones
Como se mencionó anteriormente, el conjunto de datos inicial no parece ser suficiente para responder las preguntas de agrupamiento. Por lo tanto, se decide utilizar una base de datos con información de los inmuebles de las instituciones de educación superior y que contiene varios identificadores en común con nuestro conjunto inicial, facilitando la unión de registros entre ambos.

Esta nueva base de datos posee diferentes pestañas, de las cuales se utilizarán las siguientes:
* Inmuebles: Contiene información sobre los inmuebles de las instituciones de educación superior, como la superficie construida, número de salas, etc. 
* Laboratorios y Talleres: Contiene información sobre la cantidad de laboratorios y talleres que posee cada institución de educación superior, incluyendo el número de computadores.
* Docentes: Esta pestaña contiene información sobre la cantidad de docentes que trabajan en cada institución de educación superior y el número de horas que trabajan, detallando también el número de docentes que trabajan en magíster y doctorado.

Con el fin de realizar los pasos propuestos en la metodología experimental, se procede a la creación de un pipeline de preprocesamiento. Esto, además de permitirnos ajustar los hiperparámetros de los algoritmos de preprocesamiento, nos permite evitar el data leakage al momento de realizar la validación cruzada.

Partiendo por el conjunto inicial y recordando el desbalance de la base de datos, se decide utilizar solo las carreras de Pregrado (83.8% de los datos). Con esto, algunas columnas del conjunto de datos "Docentes" no son relevantes, como el "N°MagísterJornadaHora" (que además tiene una tasa de nulos del 33%) o "N°DoctorJornadaHora" (con una tasa de nulos del 61%) por lo que se decide eliminarlas.

Luego, se decide eliminar las columnas que no aportan información relevante para el análisis, como el orden geográfico de la región, el idgenerocarrera y el Código SIES. En cuanto a los conjuntos de datos "inmuebles", "labs" y "docentes", se eliminaron las columnas ...

Tras esto, se procede a obtener las columnas que cuentan con una alta tasa de nulos con la finalidad de eliminarlas. Se experimentó con umbrales de 0.5, 0.6 y 0.7, obteniendo los siguientes resultados:

...

El siguiente paso realizado es el de imputar los nulos de las columnas restantes. Por el lado de los algoritmos de imputación univariante se encuentran el algoritmo SimpleImputer (que utiliza la media, mediana o moda para rellenar los nulos), mientras que por el lado de los multivariantes se está el algoritmo IterativeImputer (que utiliza un modelo de regresión para predecir los valores nulos). Adicionalmente, se cuenta con el algoritmo KNNImputer, que utiliza los k vecinos más cercanos para predecir los valores nulos. 

Aunque dentro de la metodología propuesta la etapa de imputación ocurre antes del mapeo y la aplicación de OneHotEncoder, se decidió realizarlo después de estos pasos para evitar que los valores nulos se mapearan a una categoría o se crearan columnas de nulos en el caso de OneHotEncoder. Los resultados obtenidos son los siguientes:

<!-- A diferencia de los algoritmos mencionados previamente, este algoritmo requiere mapear las columnas cualitativas nominales a valores numéricos, por lo que se debieron realizar los dos pasos siguientes acordados en la propuesta . -->

...

Como último paso del preprocesamiento, se realiza la unión de los cuatro conjuntos de datos. Para esto, se realiza un join entre el conjunto de datos inicial y los conjuntos de datos "inmuebles", "labs" y "docencia" utilizando el código de la institución y el código de la carrera. Luego, se realiza un join entre el conjunto de datos "docentes" y el conjunto de datos resultante del join anterior utilizando el código de la institución. Finalmente, se realiza un join entre el conjunto de datos resultante del join anterior y el conjunto de datos "inicial" utilizando el código de la carrera.


# Anexo

In [ ]:
from pathlib import Path
import joblib

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.pipeline import Pipeline

from pipeline import (
    DropColumns,
    DropHighNAPercentage,
    NormalizeCurrency,
    OrdinalColumnMapper,
)

pd.set_option('display.max_columns', 100)

In [ ]:
ROOT_DIR = Path.cwd().parent
DATA_DIR = ROOT_DIR / "data"
assert DATA_DIR.exists()

DATA_PATH = DATA_DIR / "base_indices_2005-2023.xlsx"

In [ ]:
# # xlsx to pkl
df = pd.read_excel(DATA_PATH)
joblib.dump(df, DATA_DIR / 'base_indices.pkl')

In [ ]:
df = joblib.load(DATA_DIR / "base_indices.pkl")
df = df.reset_index()

## Exploración

### Hito 1

In [ ]:
df.info()

In [ ]:
# Plot distribution of pregrado vs posgrado in a pie chart
conteo_pregrado = df[df['Pregrado/Posgrado'] == 'Pregrado'].shape[0]
conteo_posgrado = df[df['Pregrado/Posgrado'] == 'Posgrado'].shape[0]

# Crear un gráfico de sectores
labels = ['Pregrado', 'Posgrado']
sizes = [conteo_pregrado, conteo_posgrado]
colors = ['lightskyblue', 'lightcoral']

plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.title('Distribución de Pregrado vs Posgrado')
plt.show()

In [ ]:
df['Nombre Region'].value_counts().plot(kind='barh')
plt.ylabel('Región')
plt.xlabel('Cantidad de estudiantes')
plt.title('Cantidad de estudiantes por región')

In [ ]:
df['Tipo Carrera'].value_counts().plot(kind='barh')
plt.ylabel('Tipo de carrera')
plt.xlabel('Cantidad de estudiantes')
plt.title('Cantidad de estudiantes por tipo de carrera')

In [ ]:
df['Area Conocimiento'].value_counts().plot(kind='barh')
plt.ylabel('Área de conocimiento')
plt.xlabel('Cantidad de estudiantes')
plt.title('Cantidad de estudiantes por área de conocimiento')

In [ ]:
df['Año'] = pd.to_datetime(df['Año'], format='%Y')
df.groupby("Año")['Promedio Puntaje (promedio matemáticas y lenguaje)'].max()

In [ ]:
# Datos PSU y PTU
datos_puntaje_PSU_PTU=df[df['Año'].dt.year<2023]['Promedio Puntaje (promedio matemáticas y lenguaje)'].value_counts().sort_index().plot()
plt.xlabel("Promedio puntaje")
plt.ylabel("Cantidad de estudiantes")
plt.title("Distribución de promedio de puntaje para la PSU-PTU de matemáticas y lenguaje")
plt.show()

# Datos PAES
datos_puntaje_PAES=df[(df['Año'].dt.year>=2023)]['Promedio Puntaje (promedio matemáticas y lenguaje)'].value_counts().sort_index().plot()
plt.xlabel("Promedio puntaje")
plt.ylabel("Cantidad de estudiantes")
plt.title("Distribución de promedio de puntaje para la PAES de matemáticas y lenguaje")

plt.show()

### Hito 2

In [ ]:
df['Orden Geográfico de la Región (Norte aSur)'].value_counts()

In [ ]:
# show percentage of null values in Mención o Especialidad
df['Mención o Especialidad'].isnull().sum() / df.shape[0]

In [ ]:
# show unique values in idgenerocarrera
df['idgenerocarrera'].value_counts()

In [ ]:

df['Códgo SIES'].value_counts()

In [ ]:
# show percentage of null values in each column and sort them in descending order
nulls = df.isnull().sum() / len(df) * 100
nulls.sort_values(ascending=False)


In [ ]:
interes1 = [
    "Clasificación1",
    "Clasificación2",
    "Clasificación3",
    "Clasificación4",
    "Clasificación5",
    "Clasificación6",
]
for column in interes1:
    print(f"Valores únicos de la columna {column}: \n \n{df[column].unique()} \n ")

interes2 = [
    "Horario",
    "Tipo Programa",
    "IngresoDirecto",
    "Tipo Moneda",
    "Pregrado/Posgrado",
]
for column in interes2:
    print(f"Valores únicos de la columna {column}: \n \n{df[column].unique() } \n ")
